## Sequência de funções valor para os credores

Queremos resolver por indução retroativa um jogo com T períodos.

Estou trabalhando em como deixar esse jogo em torno do parâmetro T

Teremos duas sequências de funções valor: uma para quando apenas o credor sênior propõe e outra para quando apenas o credor júnior propõe. As funções do jogo final serão uma média ponderada destas.



In [1]:
#para mostrar todos os resultados e não apenas o último
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [2]:
#libraries
import numpy as np
from numba import jit, jitclass, float64, njit
import matplotlib.pyplot as plt
%matplotlib inline
import quantecon as qe
from scipy.stats import beta

from random import uniform #para a draw da uniforme(0,1)
import math

import time #tempo de computação


### Passo 0: definindo comandos do latex para facilitar a escrita

$%conjunto de informações de s em t$
$\newcommand{\Is}[1]{\theta_{s{#1}}, l_{s{#1}}, l_{j{#1}}}$
$%macros para facilitar a escrita de funções valor$


$%conjunto de informações de j em t$
$\newcommand{\Ij}[1]{\theta_{j{#1}}, l_{s{#1}}, l_{j{#1}}}$

$%função valor de s em t$
$\newcommand{\Ws}[1]{ W_{s{#1}} ( \Is{{#1}}) }$

$%função valor de j em t$
$\newcommand{\Wj}[1]{ W_{j{#1}} ( \Ij{{#1}}) }$

$%operador esperança de s em t. 2 argumentos: o primeiro é o período e o segundo é o termo que ela tira a esperança$

$\newcommand{\Es}[2]{\mathbb{E_{#1}^{s} \big[ {#2}  \mid ( \Is{#1} )    \big] }}$

$%minúsculo não mostra o conjunto de informação$
$\newcommand{\es}[2]{\mathbb{E_{#1}^{s} \big[ {#2}  \big] }}$

$%minúsculo não mostra o conjunto de informação$
$%final b de big para aumentar os parênteses$
$\newcommand{\esb}[2]{\mathbb{E_{#1}^{s} \bigg[ {#2}  \bigg] }}$

$%operador esperança de j em t. 2 argumentos: o primeiro é o período e o segundo é o termo que ela tira a esperança$

$\newcommand{\Ej}[2]{\mathbb{E_{#1}^{j} \big[ {#2}  \mid ( \Ij{#1} )    \big] }}$

$%minúsculo não mostra o conjunto de informação$
$\newcommand{\ej}[2]{\mathbb{E_{#1}^{j} \big[ {#2}  \big] }}$

$%minúsculo não mostra o conjunto de informação$
$%final b para aumentar os parênteses$
$\newcommand{\ejb}[2]{\mathbb{E_{#1}^{j} \bigg[ {#2}  \bigg] }}$


$%comando para usar o máximo com chaves grandes$
$\newcommand{\maximo}[1]{\max \bigg\{ #1 \bigg\}}$


In [3]:
#parâmetros do jogo, apenas para ilustração a princípio



μ = 3 #número de meses entre períodos, não entendi onde entra ainda
ρ = 0.9 # (1 - ρ) é a taxa de depreciação da empresa a cada período
β = 1 #usamos aqui a distribuição Uniforme, depois vamos mudar isto
c0 = 0.05 #custo fixo de ir para a corte
c1 = 0.015 #custo variável de ir para a corte




λj = 0.5 #probabilidade de j propor a cada turno. Não precisaremos estimar, isso virá dos dados


θ_s0 = 0.5 #habilidade inicial de s
θ_j0 = 0.5 #habilidade inicial de j

In [4]:
#informações que virão dos dados

Vmax = 100
L = 20 
L_s = 10
L_j = 10

#valores da dívida de cada jogador (virá dos dados, aqui é exemplo):
Ds = 20
Dj = 10

### Qual o ganho que o credor s tem em propor a cada período?

Suponha que ele vai propor reorganizar a firma no perído t. A reorganização demora 1 período, então o valor da firma aparece no próximo período, $V_{t+1}$. E ele precisa fazer uma estimativa sobre $\theta_{s,t+1}$ dada a informação que ele tem no período atual, que eu chamei de k: $\theta_{s,k}$:

$$ R_t (\theta_{s,k}) = V_{t+1} \bigg( \mathbb{E}_k^s \big[ \theta_{s,t+1} \mid \theta_{s,k} \big] \bigg) - L_j$$


#### Qual a estimativa da habilidade dele nos períodos à frente dada a informação dele hoje?

Essa parte depende da hipótese sobre a distribuição da habilidade.

Se supormos que a habilidade segue distribuição Uniforme, a forma fechada dela é:

Suponha $ t > k$

$$ \mathbb{E}_k^s \big[ \theta_{s,t} \mid \theta_{s,k} \big] =  \frac{\theta_{s,k}} {2^{t - k}} + \sum_{n=1}^{t - k} \frac{1}{2^{n}} $$


#### Quais os passos do algoritmo para implementar essa estratégia?

1. Calcular todos os valores de reorganização $ R_t $ de $ t \in  \{k, k+1, ..., T\}$ e também o valor de nunca reorganizar, que é $L_s$

2. Criar uma condição para checar se $\max \{R_{k}, R_{k+1}, ..., R_{T}, L_s \} = R_{k}$
    * em caso positivo, oferecer $P_s = W_{j, t+1} = L_j$ a fim de reorganizar a firma
    * em caso negativo, oferecer $P_s < W_{j, t+1} = L_j$ a fim de adiantar o jogo até o próximo período
    
3. Repetir os passos 1 e 2 até que reorganize a firma ou o jogo termine com ela liquidada

### Passo 1: número máximo de turnos

Calculado com base nos parãmetros

In [5]:
#função para calcular o máximo de turnos do jogo
def maximo_de_turnos(ρ, Vmax, L):
    
    t = 1
    
    while ρ**t * Vmax > L:
        
        t = t + 1
        
    return t
    

In [6]:
#guardando o número máxim de turnos

T = maximo_de_turnos(ρ, Vmax, L)

T

16

In [7]:
#valor máximo de reorganização da firma a cada período


#sequência de valores da firma para cada período

V = np.empty(T, dtype=np.int_)

for t in range(T):

    V[t] = ρ**t * Vmax

In [8]:
#valor de liquidação


#a dívida total é sempre a soma das dívidas
D = Ds + Dj

#o custo total é uma função do tempo

def C(t):
    Ct = c0 * D + c1 * t * D
    
    return Ct

#e os valores de liquidação também são função do tempo

def Ls(t):
    
    Lst = min(L - C(t), Ds)
    
    return Lst

def Lj(t):
    
    Ljt = min(L - C(t) - Ls(t), Dj)
    
    return Ljt
    
    


### Passo 2: Definir os arrays para guardar as funções valor de cada período

São:

* 100 slots para habilidade do jogador

* 100 slots para o lower bound do adversário

* 100 slots para o lower bound do próprio jogador no próximo período

* 2 slots para dizer se o jogador está propondo ou não
    
* T slots para marcar o período da função valor



In [10]:
#slots para cada habilidade
grid_size = 100


#a vantagem de colocar os dados assim é que se eu quiser teta_s = 0.115, basta procurar θs_vals[114]
θs_vals = np.linspace(0.01, 1, grid_size) 
θj_vals  = np.linspace(0.01, 1, grid_size)  


#teste
# θs_vals[99 - 1]

In [11]:
#arrays para o credor sênior



# primeira entrada é a habilidade verdadeira do jogador
# segunda entrada é lst
# terceira entrada é ljt

# quarta entrada indica se jogador está propondo (1) ou respondendo (0)
# última entrada é o período do jogo

Ws_array = np.zeros((100, 100, 100,2, T))

Wj_array = np.zeros((100, 100, 100,2, T))

#vou deixar a função policy separada porque ela vai ser um resultado da barganha entre os agentes



In [12]:
#uma das sugestões que encontrei no stack overflow foi de dividir a matriz grandona em várias matrizes pequenas
#vou tentar com 100x100x100 primeiro, depois eu adapto o código

# Ws_array1 = np.empty((100, 100, 100,T))
# Ws_array2 = np.empty((100, 100, 100,T))
# Ws_array3 = np.empty((100, 100, 100,T))
# Ws_array4 = np.empty((100, 100, 100,T))
# Ws_array5 = np.empty((100, 100, 100,T))
# Ws_array6 = np.empty((100, 100, 100,T))
# Ws_array7 = np.empty((100, 100, 100,T))
# Ws_array8 = np.empty((100, 100, 100,T))
# Ws_array9 = np.empty((100, 100, 100,T))


Populando o array com os valores de liquidação, que independem das habilidades ou lower bounds



In [13]:
Ws_array[:,:,:, :,(T-1)] = Ls(T)

Wj_array[:,:,:, :,(T-1)] = Lj(T)

In [14]:
#funções para achar os valores nas matrizes

def Ws_find(teta_st, lst, ljt, policy, t):
    
    
    #para achar teta_st = 0.99, buscamos: θs_vals[99 - 1]
    #como os valores serão em casas decimais, temos que multiplicar por 100
    vteta_st = int(100 * teta_st - 1)
    
    vlst = int(100 * lst - 1)
    
    vljt = int(100 * ljt - 1)
    
    return Ws_array[vteta_st, vlst, vljt,policy, t]
    
    

    
def Wj_find(teta_jt, lst, ljt, policy, t):
    
    
    vteta_jt = int(100 * teta_jt - 1)
    
    vlst = int(100 * lst - 1)
    
    vljt = int(100 * ljt - 1)
    
    return Ws_array[vteta_jt, vlst, vljt, policy, t]



def find(y):
    
    x = 100*y - 1
    
    #transformando em int para usar como índice nas matrizes
    
    x = int(x)
    
    return x


### Passo 3: fazer o cálculo das funções valor em T-1, T-2, ..., 1.

### Função para tirar um draw da distribuição Beta


Vamos usar o método da amostragem da inversa da CDF (https://en.wikipedia.org/wiki/Inverse_transform_sampling_method). Outra referência que usei foi: https://blogs.sas.com/content/iml/2013/07/22/the-inverse-cdf-method.html#:~:text=The%20exponential%20distribution%20has%20probability,log(1%E2%80%93u).

A CDF da Beta é 

$$ F_{\beta} ( \theta_{t+1} \mid \theta_{t} ) = 1 - \frac{ (1 - \theta_{t+1})^\beta}{ (1 - \theta_{t})^\beta }, \, \, \theta_{t} \leq \theta_{t+1} \leq 1, \, \beta \geq 1$$

Para invertê-la, basta procurarmos o valor de x tal que $F(x) = u$, onde u é uma retirada da distribuição Uniforme(0,1).

Fazendo os cálculos, esse valor de x é (ou $\theta_{t+1}$, no caso)


$$ \theta_{t+1} =  1 - exp \bigg\{ \frac{1}{\beta} \big[  log (1 - u) + \beta * log(1 - \theta_{t}) \big] \bigg\} $$


In [153]:
#código para tirar draw da distribuição beta

# def draw_beta(info_hoje):
    
#     if(info_hoje == 1):
#         return 1
#     else:
    
#         u = uniform(0, 1)
#         x = 1 - math.exp( (1/β) * (math.log(1-u) + β*math.log(1-info_hoje)) )

#         return x




#testando

# draw_beta(0.5)



#teste com draw da função UNIFORME
def draw_beta(info_hoje):
    u = uniform(info_hoje, 1)
    return u


# testando
# draw_beta(1)

1.0

Função para tirar o valor esperado do teta amanhã, dada a informação hoje.

É apenas a média de vários draws da função beta

In [147]:
# def expec_beta(info_hoje):
    

#     #para calcular o valor esperado UM PERÍODO À FRENTE, vamos tirar 1000 draws disso e fazer a média
#     #se estiver correto, expec_beta(0.5, 1) deve ser próximo de 0.75, pois Beta = 1 é Uniforme
#     beta_vals = []

#     for t in range(100):
#         beta_amanha = draw_beta(info_hoje)

#         beta_vals.append(beta_amanha)
        
#     return round(np.mean(beta_vals),2)

# #testando
# # expec_beta(0.5)



#teste com draw da função UNIFORME
def expec_beta(info_hoje):
    
    summation = 1/2
    
    resultado = info_hoje/2 + summation
    
    return round(resultado,2)

#testando
# expec_beta(0.5)

### Como obter o valor esperado da função valor no período seguinte?

Multiplicando a coluna correta de pmf pela coluna correta de Ws_array


Agora que temos a pdf, podemos fazer assim:


$$ \es{t}{ \Ws{t+1} } = \sum_{\theta_{t+1}=0.01}^{1.00} prob(\Is{t+1}) * \Ws{t+1}$$


Na prática, já saberemos $l_{s,t+1}$ e $l_{j,t+1}$, então poderemos fixar esses valores


$$ \es{t}{ \Ws{t+1} } = \sum_{\theta_{t+1}=0.01}^{1.00} prob(\theta_{t+1}) * [ \Ws{t+1} \mid l_{s,t+1}, l_{j,t+1} ]$$


Vamos fazer essa soma usando multiplicação de vetores:

* aproveitaremos que dá pra saber os lower bounds do período seguinte a cada situação, então a única incerteza é sobre teta
* multiplicaremos pmf (vetor linha)
* pelo vetor coluna W correspondente


Como achar a entrada correspondente na matriz?
Por exemplo, achar qual entrada corresponde a lst =  0.58. Basta escrever 57 na entrada correspondente. Ou usar a função find(0.58).




In [154]:
#binning da pdf beta

#gerando 1000 draws

def bin(info_hoje, ndraws):
    
    beta_vals = []

    for k in range(ndraws):
        beta_amanha = draw_beta(info_hoje)

        beta_vals.append(beta_amanha)
    
    #conta cria os bins e conta quantos valores estão dentro deles
    teta_bins = np.zeros(len(θj_vals))  
    
    for t in range(len(teta_bins)-1):
        
        pre = (θj_vals[t-1]+θj_vals[t])/2
               
        pos = (θj_vals[t]+θj_vals[t+1])/2
        
        if(t==0):
            pre = 0
        else:
            if(t==len(teta_bins)-1):
                pos = 1
                
       
        
        
        for b in beta_vals:
            
            if(b > pre and b < pos):
                
                teta_bins[t] += 1
    
    return teta_bins/ndraws
        
        
        
        
#dá algum erro porque a soma não retorna 1 quando Beta = 1
# sum(bin(0.5, 1, 1000))



In [156]:
#vamos gerar uma matriz com 100 linhas e 100 colunas
#cada coluna vai representar as probabilidades de teta_amanhã dado teta hoje
#a linha 1 significa que teta_hoje é 0.01
#assim, a linha 1 tem as probabilidades de teta_amanhã dado que teta_hoje é 0.01




#probability mass function
pmf = np.zeros((100,100))

#exemplo para ilustrar
# pmf[0,] = bin(θj_vals[0], β, 1000)


#populando a pmf:

for t in range(len(θj_vals)-1):
    
    pmf[t,:] = bin(θj_vals[t], 1000)

#quais as probabilidades de teta_amanhã se eu sei que teta_hoje = 0.5?

# sum(pmf[54,:])

0.9890000000000005

In [158]:
#teste eliminando valores abaixo de teta_hoje


pmf[54,:]

pmf[54, 54:100]

array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.012, 0.025, 0.022, 0.026, 0.016, 0.029, 0.019, 0.017, 0.02 ,
       0.02 , 0.027, 0.017, 0.025, 0.016, 0.017, 0.019, 0.02 , 0.024,
       0.023, 0.028, 0.022, 0.018, 0.02 , 0.02 , 0.024, 0.028, 0.028,
       0.02 , 0.024, 0.019, 0.026, 0.019, 0.028, 0.02 , 0.015, 0.033,
       0.034, 0.017, 0.02 , 0.019, 0.027, 0.02 , 0.018, 0.032, 0.016,
       0.   ])

array([0.012, 0.025, 0.022, 0.026, 0.016, 0.029, 0.019, 0.017, 0.02 ,
       0.02 , 0.027, 0.017, 0.025, 0.016, 0.017, 0.019, 0.02 , 0.024,
       0.023, 0.028, 0.022, 0.018, 0.02 , 0.02 , 0.024, 0.028, 0.028,
       0.02 , 0.024, 0.019, 0.026, 0.019, 0.028, 0.02 , 0.015, 0.033,
       0.034, 0.017, 0.02 , 0.019, 0.027, 0.02 , 0.018, 0.032, 0.016,
       0.   ])

In [162]:
#valor esperado das funções valor, já organizado

def Ews(teta_hoje, ls_amanha, lj_amanha, t):
    
    teta_hoje = find(teta_hoje)
    
    ls_amanha = find(ls_amanha)
    lj_amanha = find(lj_amanha)
    
    
    
    Esp_respondendo = sum(pmf[teta_hoje, teta_hoje:100] * Ws_array[teta_hoje:100,ls_amanha, lj_amanha, 0, (t-1) ])
    Esp_propondo = sum(pmf[teta_hoje, teta_hoje:100] * Ws_array[teta_hoje:100,ls_amanha, lj_amanha, 1, (t-1) ])
    
    
    Esp = λj * Esp_respondendo + (1 - λj) * Esp_propondo
    
    return Esp


def Ewj(teta_hoje, ls_amanha, lj_amanha, t):
    
    teta_hoje = find(teta_hoje)
    
    ls_amanha = find(ls_amanha)
    lj_amanha = find(lj_amanha)
    
    
    
    Esp_respondendo = sum(pmf[teta_hoje, teta_hoje:100] * Wj_array[teta_hoje:100,ls_amanha, lj_amanha, 0, (t-1) ])
    Esp_propondo = sum(pmf[teta_hoje, teta_hoje:100] * Wj_array[teta_hoje:100,ls_amanha, lj_amanha, 1, (t-1) ])
    
    #aqui invertemos a ordem porque j propõe com chance lambda_j
    Esp = λj * Esp_propondo  + (1 - λj) * Esp_respondendo
    
    return Esp


#teste


# start = time.process_time()


# Ews(0.5, 0.3, 0.2, 16)
# print(time.process_time() - start)

# sum(pmf[49, :] * Ws_array[49,29, 19, 1, 15])


11.2435

0.0


11.2435

#### Primeiro vamos explicar os cálculos, depois a gente explica a notação de matriz mais adiante


A primeira parte da função valor de s é caso ele venha a propor:


 $$ S_1 (\Is{1}) = \max \bigg\{ L_{s1}, V \es{1}{\theta_s} - \es{1}{W_j}, \es{1}{W_s}
                    \bigg\} $$



In [163]:
#payoff de S quanto ele tem o direito de propor em t

 
#Tem que ter preparado o terreno antes calculando as funções valor no período seguinte

def St(teta_st, lst, ljt, t):
    
    #update do lower bound:
    
    #ls_next é a habilidade dele hoje pois s revela esta quando propõe
    
    ls_next = teta_st
    
      
    
    #quanto s pagará para j neste período? A esperança da função valor de j amanhã, conforme as informações de j hoje.
    
    Pst = Ewj(ljt, lst, ljt, t+1)
    
    
    #lj_next depende do cutoff de j
    
    #algoritmo para calcular cutoff
    
    cjt = ljt
    
    
    #flag é vai parar o loop se demorar muito
    flag = 1
    tol = 0.01
    print
    
    while (Pst - Ewj(cjt, ls_next, cjt, t+1) > tol and flag < 10000):
        cjt = cjt + 0.01
        flag = flag + 1
        
    
    #lj_next é o máximo entre o lower bound de j hoje e o cutoff de j
    lj_next = max(ljt, cjt)
    
    #esperança da própria habilidade no próximo período
    θs_next = expec_beta(teta_st)
    
    #OLHAR ESSA PARTE COM CUIDADO, CONFERIR OS ARGUMENTOS DAS FUNÇÕES NO PERÍODO SEGUINTE
    St = [Ls(t), V[t] * θs_next - Pst, Ews(θs_next, ls_next, lj_next, t+1)]
          
    
    policy_St = np.argmax(St)
    
    
    #retorna um vetor com dois elementos: a função valor e a política ótima
    return St[policy_St]


    

    


Quando j propõe é análogo

In [164]:
#payoff de S quanto ele tem o direito de propor em t

 
#Tem que ter preparado o terreno antes calculando as funções valor no período seguinte

def Jt(teta_jt, lst, ljt, t):
    
    #update do lower bound:
    
    #ls_next é a habilidade dele hoje pois s revela esta quando propõe
    
    lj_next = teta_jt
    
      
    
    #quanto s pagará para j neste período? A esperança da função valor de j amanhã, conforme as informações de j hoje.
    
    Pjt = Ews(lst, lst, ljt, t+1)
    
    
    #lj_next depende do cutoff de j
    
    #algoritmo para calcular cutoff
    
    cst = lst
    
    
    #flag é vai parar o loop se demorar muito
    flag = 1
    tol = 0.01
    
    while (Pjt - Ews(cst, cst, lj_next, t+1) > tol and flag < 10000):
        cst = cst + 0.01
        flag = flag + 1
        
    
    #lj_next é o máximo entre o lower bound de j hoje e o cutoff de j
    ls_next = max(lst, cst)
    
    #esperança da própria habilidade no próximo período
    θj_next = expec_beta(teta_jt)
    
    #OLHAR ESSA PARTE COM CUIDADO, CONFERIR OS ARGUMENTOS DAS FUNÇÕES NO PERÍODO SEGUINTE
    Jt = [Lj(t), V[t] * θj_next - Pjt, Ewj(θj_next, ls_next, lj_next, t+1)]
          
    
    policy_Jt = np.argmax(Jt)
    
    
    #retorna um vetor com dois elementos: a função valor e a política ótima
    return Jt[policy_Jt]


    

    


A segunda parte da função valor é se se for chamado a responder:
    
$s_1 (\Is{1}) = $
    $ \esb{1}{ Prob ({j liquidar})  \maximo{L_{s1}, V \es{1}{\theta_s} - L_{j1}} } $
    $ + \esb{1}{ Prob ({j reorganizar})   \maximo{\ej{1}{W_s}, \es{1}{W_s}}    }$
 
 
 Sendo que
 
 $ \Es{1}{Prob (j liq em T-1)}$
 
  $$ = Prob \bigg(L_{j1} > \max \{V \es{1}{ \ej{1}{\theta_j} } - \es{1}{ \ej{1}{W_s} }, \es{1}{ \ej{1}{W_j}  }\} \bigg)$$
  
  
 Repare que $L_{j1} > L{j}$ e que $\es{1}{\theta_{j}}$ cresce em $\theta_{j}$. Então deve existir um threshold de reorganização em t = T-1, $\phi_{j1}$, tal que: 

$$V \phi_{j1} - L_{s} = L{j1} $$

$$ \phi_{j1} = \frac{ L_{j1} + L_{s} }{V} $$

Então se $\es{1}{\theta_{j}} > \phi_{j1}$, j reorganiza a firma. E j liquida a firma caso contrário.


A pergunta é: qual a probabilidade de que $\theta_{j}$ tome um valor menor ou igual a $\phi_{j1}$ ? Isso pode ser respondido usando a \textit{prior} do jogador, a função CDF da distribuição Beta:

$$ \Es{1}{Prob (\textnormal{j liq em T-1})} = F_{\beta} \big( \phi_{j1} \mid l_{j1} \big) $$
 


Na probabilidade de liquidar no período seguinte, temos a seguinte expressão:

$ Prob \bigg( L_{j,t} > \maximo{V_{t+1} \es{t}{\theta_{j,t+1} }  - \es{t}{ \ej{t}{ W_{s,t+1}  } }  ,  \es{t}{ \ej{t}{ W_{j,t+1}  } } } \bigg)$

Onde os termos dentro das expectativas iteradas são calculados com base apenas nos lower bounds de hoje.

1. Vamos considerar o caso em que a esperança de j sobre a sua própria função valor  é o máximo dos argumentos

2. Vamos calcular o threshold para o caso onde ainda pode compensar reorganizar a firma

In [165]:
#vamos criar os componentes da função st

#não preciso me preocupar com if t == T, pois esta condição estará nas funções valor Ws e Wj

def Prob_s(teta_st, lst, ljt, t):
    
    #probabilidade de j liquidar em t, dado ljt 
    #sob a ótica de s, daí o subscrito _s
    
    #caso onde a esperança de j sobre a própria função é o máximo dos argumentos
    if (Ewj(ljt, lst, ljt, t+1) > V[t] * 1 - Ews(lst, lst, ljt, t+1) ):
        
        #se a esperança da função valor amanhã for maior ou igual ao valor de liquidar,
        #a chance de liquidar é zero
        
        if(Ewj(ljt, lst, ljt, t+1) >= Lj(t)):
            return 0
        else: 
            return 1
        
    #caso onde ainda compensa para j reorganizar a firma
    else:

        #threshold para j tentar liquidar
        
        #DÚVIDA SE AQUI EU PRECISO USAR LJ_NEXT E LS_NEXT.
        #ACHO QUE SIM, VOU CONFERIR DEPOIS
        ϕjt = ( Lj(t) + Ews(lst, lst, ljt, t+1) ) / V[t]

        Prob_st = 1 - ((1 - ϕjt)**β)/((1 - ljt)**β)

        #aqui tem que ser no mínimo a igual a zero, apenas por segurança
        Prob_st = max(0, Prob_st)

        return Prob_st



A probabilidade para j é análoga

In [166]:
#vamos criar os componentes da função st

#não preciso me preocupar com if t == T, pois esta condição estará nas funções valor Ws e Wj

def Prob_j(teta_jt, lst, ljt, t):
    
    #probabilidade de s liquidar em t, dado lst 
    #sob a ótica de j, daí o subscrito _j
    
    #caso onde a esperança de s sobre a própria função é o máximo dos argumentos
    if (Ews(lst, lst, ljt, t+1) > V[t] * 1 - Ewj(ljt, lst, ljt, t+1) ):
        
        #se a esperança da função valor amanhã for maior ou igual ao valor de liquidar,
        #a chance de liquidar é zero
        
        if(Ews(lst, lst, ljt, t+1) >= Ls(t)):
            return 0
        else: 
            return 1
        
    #caso onde ainda compensa para j reorganizar a firma
    else:

        #threshold para j tentar liquidar
        
        #DÚVIDA SE AQUI EU PRECISO USAR LJ_NEXT E LS_NEXT.
        #ACHO QUE SIM, VOU CONFERIR DEPOIS
        ϕst = ( Ls(t) + Ewj(ljt, lst, ljt, t+1) ) / V[t]

        Prob_jt = 1 - ((1 - ϕst)**β)/((1 - lst)**β)

        #aqui tem que ser no mínimo a igual a zero, apenas por segurança
        Prob_jt = max(0, Prob_jt)

        return Prob_jt



Payoff de s caso ele responda à uma proposta de liquidação de j

In [167]:
#Payoff de s caso ele responda à uma proposta de liquidação de j

def s_liq(teta_st, lst, ljt, t):
    
    #update dos lower bounds não é necessário aqui
    #é a unica parte da função que independe das funções valor no período seguinte
    
    st_liq = [Ls(t), V[t] * expec_beta(teta_st) - Lj(t) ]

    policy_st_liq = np.argmax(st_liq)


    st_liq = st_liq[policy_st_liq]

    return (st_liq)




Payoff de j caso ele responda à uma proposta de liquidação de s

In [168]:
#Payoff de j caso ele responda à uma proposta de liquidação de s


def j_liq(teta_jt, lst, ljt, t):
    
    #update dos lower bounds não é necessário aqui
    #é a unica parte da função que independe das funções valor no período seguinte
    
    jt_liq = [Lj(t), V[t] * expec_beta(teta_jt) - Ls(t) ]

    policy_jt_liq = np.argmax(jt_liq)


    jt_liq = jt_liq[policy_jt_liq]

    return (jt_liq)




payoff caso s responda à uma proposta de reorganização de j


In [169]:
#payoff caso s responda à uma proposta de reorganização de j

def s_reorg(teta_st, lst, ljt, t):
    
    #update dos lower bounds
    
    
    #neste caso quem revela a informação é j,
    #vamos ter que colocar o update do lower bound na estrutura da barganha do jogo depois
    lj_next = expec_beta(ljt)
    
    #algoritmo para calcular cutoff
    
    cst = lst
    
    #flag é vai parar o loop se demorar muito
    flag = 1
    tol = 0.01
    #vou usar f() porque a habilidade em f() cresce 0.1 por período
    #se cst crescer 0.1, significa que a função funcionou bem
    
    Pjt = Ews(lst, lst, ljt, t+1)
    
    
    while (Pjt - Ews(cst, cst, lj_next, t+1) > tol and flag < 10000):
        cst = cst + 0.1
        flag = flag + 1
        
    
    #lj_next é o máximo entre o lower bound de j hoje e o cutoff de j
    ls_next = max(lst, cst)
    
    
    
    st_reorg = [Ews(ls_next, ls_next, lj_next, t+1), Ews(teta_st, ls_next, lj_next, t+1)]

    policy_st_reorg = np.argmax(st_reorg)


    st_reorg = st_reorg[policy_st_reorg]
    
    return (st_reorg)



payoff caso j responda à uma proposta de reorganização de s


In [170]:
#payoff caso j responda à uma proposta de reorganização de s

def j_reorg(teta_jt, lst, ljt, t):
    
    #update dos lower bounds
    
    
    #neste caso quem revela a informação é s,
    #vamos ter que colocar o update do lower bound na estrutura da barganha do jogo depois
    ls_next = expec_beta(lst)
    
    #algoritmo para calcular cutoff
    
    cjt = ljt
    
    #flag é vai parar o loop se demorar muito
    flag = 1
    tol = 0.01
    #vou usar f() porque a habilidade em f() cresce 0.1 por período
    #se cst crescer 0.1, significa que a função funcionou bem
    
    Pst = Ewj(ljt, lst, ljt, t+1)
    
    
    while (Pst - Ewj(cjt, ls_next, cjt, t+1) > tol and flag < 10000):
        cjt = cjt + 0.1
        flag = flag + 1
        
    
    #lj_next é o máximo entre o lower bound de j hoje e o cutoff de j
    lj_next = max(ljt, cjt)
    
    
    
    jt_reorg = [Ewj(lj_next, ls_next, lj_next, t+1), Ewj(teta_jt, ls_next, lj_next, t+1)]

    policy_jt_reorg = np.argmax(jt_reorg)


    jt_reorg = jt_reorg[policy_jt_reorg]
    
    return (jt_reorg)



Finalmente, calculando o valor da função inteira quando os credores estão respondendo a uma proposta

In [171]:
def st(teta_st, lst, ljt, t):
    
    #agregando todas as funções até agora
    
    st = (Prob_s(teta_st, lst, ljt, t)) * s_liq(teta_st, lst, ljt, t) + (1 - Prob_s(teta_st, lst, ljt, t)) * s_reorg(teta_st, lst, ljt, t)
    
    return (st)

In [172]:
def jt(teta_jt, lst, ljt, t):
    
    #agregando todas as funções até agora
    
    jt = (Prob_j(teta_jt, lst, ljt, t)) * j_liq(teta_jt, lst, ljt, t) + (1 - Prob_j(teta_jt, lst, ljt, t)) * j_reorg(teta_jt, lst, ljt, t)
    
    return (jt)

In [173]:
#testando rapidamente os cálculos
#tem que ser período igual a 15 porque é o penúltimo período


# St(0.5,0.4, 0.3, 15)
# Jt(0.5,0.4, 0.3, 15)
# st(0.5,0.4, 0.3, 15)
# jt(0.5,0.4, 0.3, 15)

#todos eles retornam números, então passam no primeiro teste
#depois vamos olhar o código de cada um com cuidado para ver se não tem erro


O que a gente sabe até agora? 

* o valor das funções valor no último período (valor de liquidação)

* calcular as funções valor no penúltimo período, usando St, Jt, st e jt.

Falta apenas a gente popular as matrizes com um loop começando de trás para frente

### Passo 4: populando os arrays usando loops

Vamos relembrar a notação de matriz





#### Como é a notação dessa matriz, na prática?


$ W_{st} (\Is{1})   =$ Ws_array[ $\theta_{st}, l_{st}, l_{jt}, t$] 

Exemplo: 

Ws_array[ $0.5, 0.3,  0.2, 1, 14$] = St(0.5, 0.3,0.2,15)

Traduzindo para St:

* St maiúsculo porque a quarta entrada igual a 1 indica que s propõe

* 0.5, 0.3, 0.2 são os mesmos valores dados pelo conjunto de informação de s

* t = 15 em Ws_array porque a entrada 14 do array se refere ao período 15



Loop para popular as matrizes das funções valor no penúltimo período.

É um ensaio antes de rodar o jogo inteiro

In [174]:
#populando a matriz Ws_Array no penúltimo período quando s propõe

ls_vals = θs_vals
lj_vals = θj_vals




start = time.process_time()


#é um loop com 3 camadas, uma para cada informação




#usamos a função find() para achar a entrada correspondente no array

#usei zip porque ls_amanha será igual a teta_s hoje, pois s revela sua habilidade
for i, j in zip(θs_vals,ls_vals):
    for k in lj_vals:
        Ws_array[find(i), find(j), find(k),1,14] = St(i, j, k, 15)

print(time.process_time() - start)

#demorou 29s para fazer esse loop. inicialmente

#14/07 demorou 1.39s considerando expectativas com função uniforme e 
#eliminando multiplicação por zero no cálculo do valor esperado em Ews e Ewj


1.390625


In [175]:
#testando loop mais completo
start = time.process_time()

for i in θs_vals:
    print(time.process_time() - start)
    for j in ls_vals:
        for k in lj_vals:
            Ws_array[find(i), find(j), find(k),1,14] = St(i, j, k, 15)
            


            
print(time.process_time() - start)


0.0
1.390625
2.640625
3.890625
5.09375
6.25
7.453125
8.78125
10.015625
11.28125
12.5
13.78125
14.921875
16.140625
17.328125
18.703125
20.0
21.203125
22.359375
23.578125
25.09375
26.375
27.640625
28.9375
30.5625
31.921875
33.109375
34.40625
36.15625
37.421875
38.53125
39.734375
41.078125
42.296875
43.53125
44.8125
46.09375
47.359375
48.625
49.75
51.109375
52.359375
53.484375
54.734375
55.9375
57.25
58.46875
59.71875
60.875
62.0625
63.171875
64.265625
65.375
66.625
67.78125
68.9375
70.0
71.203125
72.390625
73.484375
74.609375
75.75
76.9375
78.15625
79.28125
80.34375
81.46875
82.625
83.765625
84.9375
86.09375
87.21875
88.4375
89.53125
90.625
91.75
92.796875
93.96875
95.265625
96.375
97.421875
98.5625
99.671875
100.734375
101.828125
102.875
104.03125
105.078125
106.140625
107.234375
108.328125
109.453125
110.5
111.546875
112.515625
113.53125
114.625
115.84375
116.921875
117.921875
118.96875


In [182]:
#testando
Ws_array[find(0.2),:,find(0.70),1, 14]

array([12., 12., 12., 12., 12., 12.,  0., 12., 12., 12., 12., 12., 12.,
       12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12.,
       12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12.,
       12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12.,
       12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12.,
       12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12.,
       12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12.,
       12., 12., 12., 12., 12., 12., 12., 12., 12.])

# Próximos passos



* HIGIENIZAR O CÓDIGO, DEIXAR ORGANIZADO!

* ~criar função de esperança baseada na função beta~
    * expec_beta(info_hoje) retorna o valor esperado de teta amanhã




* ~discretizar a beta para obter a esperança das funções valor no período seguinte~



* ~fazer Js_val~

* ~guardar resultados de Js_val e Ws_val do último período numa  matriz 4D~

* ~pensar numa função Js_next para achar o valor de Js no próximo período baseado nos parâmetros deste período~
    * regra para atualização de lst
    * regra para atualização de ljt
    * expectativa da habilidade no período seguinte

* ~pensar na forma recursiva do jogo~
    * acho que as funções probabilidade também têm que considerar o update do lower bound, pois elas fazem parte do cenário onde o jogador não propõe


* checar todas as funções de cálculo do jogo (Passo 3)


* considerar a estrutura de negociação do jogo

* aumentar a precisão das estimativas do jogo
    * algoritmo para cálculo do cutoff pode ter mais casas decimais
    
    
 * fazer uma estrutura mais enxuta, com uma função que tome como argumento se é s ou j. Ao invés de criar St, Jt, st, jt...
    


* passos finais
    * replicar os gráficos do artigo de referência


### Conferir novamente

1. Se preciso usar ls_next e lj_next nas funções Prob_s e Prob_j

2. Se o valor esperado da função para calcular os thresholds cst e cjt estão corretos

3. A função pmf não está somando um, tem que verificar onde está o erro

